since allmusic.com can't be scraped as usual (javascript), the relevant parts have to be copied manually.

In [91]:
import pprint
from bs4 import BeautifulSoup as bs
pp = pprint.PrettyPrinter(indent=4)

**use top tracks from allmusic:**

In [142]:
style = 'Deep Funk'
import os
folder = 'data'
filename = f'allmusic-{style}.html'
with open(os.path.join(folder, filename)) as f:
    soup = bs(f)
print(soup.prettify())

<section class="song-highlights">
 <h2 class="headline">
  Deep Funk Song Highlights
 </h2>
 <table>
  <thead>
   <tr>
    <th class="title sortable" data-sort-default="asc">
     <a href="#" title="Click to sort">
      Title/Composer
     </a>
    </th>
    <th class="performer sortable" data-sort-default="asc">
     <a href="#" title="Click to sort">
      Performer
     </a>
    </th>
    <th class="stream">
     Stream
    </th>
   </tr>
  </thead>
  <tbody>
   <tr>
    <td class="title" data-sort-value="Hang Up We Don't Need">
     <a href="https://www.allmusic.com/song/hang-up-we-dont-need-mt0013289197">
      Hang Up We Don't Need
     </a>
     <div class="composer">
     </div>
    </td>
    <td class="performer" data-sort-value="Bobby Byrd">
     <a href="https://www.allmusic.com/artist/bobby-byrd-mn0000059101">
      Bobby Byrd
     </a>
    </td>
    <td class="stream">
    </td>
   </tr>
   <tr>
    <td class="title" data-sort-value="I Love You! I Love You Completely">
  

In [143]:
track_soup = soup.find('tbody')
tracks = []
for row in track_soup.find_all('tr'):
    track = {}
    track['title'] = row.find('td', class_='title')['data-sort-value']
    track['artist'] = row.find('td', class_='performer')['data-sort-value']
    tracks.append(track)
tracks[0:3]

[{'title': "Hang Up We Don't Need", 'artist': 'Bobby Byrd'},
 {'title': 'I Love You! I Love You Completely', 'artist': 'Exit 9'},
 {'title': 'Sweet & Tangy', 'artist': "The J.B.'s"}]

In [144]:
import os
import spotify

sp = spotify.login(scope='playlist-read-private playlist-modify-private')
sp.current_user()['display_name']   # check if login successful

'Mehmet Birgi'

In [145]:
# get the playlist to edit
playlist_id, is_new = spotify.get_playlist_by_name(sp, 
        f"{style}", 
        create_if_none=True)
if playlist_id and is_new:
    sp.user_playlist_change_details(sp.me()['id'], playlist_id, 
                                    description=f"{style} song highlights from allmusic.com")
    print(f"Created new playlist with ID {playlist_id}")
elif playlist_id:
    print(f"Found existing playlist with ID {playlist_id}")
else:
    print("Something went wrong :-(")

Created new playlist with ID 3Z88R9JITTTARqxRoyfSqp


In [146]:
tracks_to_add = []
for track in tracks:
    q = f"{track['title']} {track['artist']}"
#     print(f"searching for: {q}")
    results = sp.search(q, type='track', market='CH')['tracks']['items']
    if results:
        spotify_id = results[0]['id']
        print(f"found: {q} - {spotify_id}")
        tracks_to_add.append(spotify_id)
    else:
        print(f"not found: {q}")
print(f"Done, {len(tracks_to_add)} tracks found")

found: Hang Up We Don't Need Bobby Byrd - 7DT6KqAWW8fHJfWuHddxPt
found: I Love You! I Love You Completely Exit 9 - 6izZJJsAFod5XAivYYtf9r
not found: Sweet & Tangy The J.B.'s
found: What's a Telephone Bill? Bootsy Collins - 5rMw53xSX8BvYiZPRVQ8he
found: 40 Acres and a Mule Sound Experience - 5AU95bRMkW5QKaBSEUjnJo
found: Shades of Brown Marva Whitney - 5LwcW7X6wZAHMnMuztJWem
not found: Looking for Somebody to Love Lee Fields
not found: (I Can't Get No) Satisfaction Incredible Bongo Band
not found: Time Is Gonna Catch You Black Heat
found: Man That I Am The Gaturs - 3HLn6ROrAG3aiz1bKY75As
not found: S.T.P. (Slinging, Teaching and Preaching) Kay-Gee's
not found: Chick a Boom Breakdown
not found: Let Me Be Your Only Man Damn Sam the Miracle Man & the Soul Congregation
found: Red Moon Fugi - 2nmYYl2ewDkaZrUPOYaApG
found: Me and You Allen Toussaint - 0fDTaVTz2AWJk81sHpGFa1
not found: Hang Down Your Head, Tom Dooley Your Tie's Caught in Your Zipper Incredible Bongo Band
found: Ain't No Sunshi

In [147]:
# add tracks to playlist
spotify.add_tracks(sp, playlist_id, tracks_to_add, skip_duplicates=True)

Playlist has 0 existing tracks
Skipping duplicates: True
36 tracks to be added
36 tracks added
OK


## Second Method

**use artist highlights from allmusic to get top tracks for them in spotify:**

In [8]:
import os
folder = 'data'
filename = 'allmusic-clubjazz-artists.html'
with open(os.path.join(folder, filename)) as f:
    soup = bs(f)
print(soup.prettify())

<section class="artist-highlights">
 <h2 class="headline">
  Clubjazz Artists Highlights
  <nav class="view-type">
   <a class="full active" data-href="/style/ajax/clubjazz-ma0000011890/artists/gridview" href="#">
    grid
   </a>
   <a class="condensed" data-href="/style/ajax/clubjazz-ma0000011890/artists/listview" href="#">
    list
   </a>
  </nav>
 </h2>
 <div class="artist-highlights-container">
  <div class="artist-highlight">
   <a data-hasqtip="0" data-tooltip='{"id":"MN0000772721"}' href="/artist/incognito-mn0000772721" oldtitle="Incognito" title="">
    <div class="cropped-image crop-image-borders" style="width:185px;height:185px;">
     <img alt="Incognito" class="lazy" data-original="https://cps-static.rovicorp.com/3/JPG_250/MI0004/743/MI0004743719.jpg?partner=allrovi.com" height="186" src="https://cps-static.rovicorp.com/3/JPG_250/MI0004/743/MI0004743719.jpg?partner=allrovi.com" style="left: -4px; display: inline;" width="195"/>
     <noscript>
      <img alt="Incognito" h

In [10]:

artists = set()
artist_tags = soup.find_all('div', class_='artist')
for tag in artist_tags:
    artists.add(tag.text.strip())
artists

{'Caro Emerald',
 'De-Phazz',
 'Gilles Peterson',
 'Incognito',
 'Jaga Jazzist',
 'Jazzanova',
 'Mario Biondi',
 'Mocean Worker',
 'Mocky',
 'Quantic',
 'Salvador Santana',
 'Soil & "Pimp" Sessions'}

In [25]:
artist_ids = set()
for artist in artists:
    q = f'artist:"{artist}"'
    artist_ids.add(sp.search(q, limit=1, type='artist', market='CH')['artists']['items'][0]['id'])
len(artist_ids)

12

In [26]:
track_ids = []
for artist_id in artist_ids:
    track_ids.extend([track['id'] for track in sp.artist_top_tracks(artist_id)['tracks'][0:3]])
len(track_ids)

36

In [27]:
# get the playlist to edit
playlist_id, is_new = spotify.get_playlist_by_name(sp, 
        f"Allmusic - Clubjazz Artists - Popular Tracks", 
        create_if_none=True)
if playlist_id and is_new:
    print(f"Created new playlist with ID {playlist_id}")
elif playlist_id:
    print(f"Found existing playlist with ID {playlist_id}")
else:
    print("Something went wrong :-(")

Created new playlist with ID 0kWG4VR9NbUG9co1EcogkG


In [28]:
# add tracks to playlist
spotify.add_tracks(sp, playlist_id, track_ids, skip_duplicates=True)

Playlist has 0 existing tracks
Skipping duplicates: True
36 tracks to be added
36 tracks added
OK
